In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.tools import DuckDuckGoSearchRun
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv

load_dotenv()

search = DuckDuckGoSearchRun()

template = """turn the following user input into a search query for a search engine:

{input}"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI(model='gpt-3.5-turbo')

chain = prompt | model | StrOutputParser() | search

chain.invoke({"input": "I'd like to figure out what games are tonight"})


"What time are the NFL games tonight? Date: Monday, Dec. 11 Time: 8:15 p.m. ET | 5:15 p.m. PT Dolphins vs. Titans and Packers vs. Giants will both kick off around 8:15 ... How to live stream NBA games tonight. Pelicans vs Thunder and Clippers vs Lakers will air on ESPN. Viewers can also stream NBA games on Sling TV. Fans in the U.S. can watch the biggest games of ... Here's all you need to know to watch the NFL's Monday night's doubleheader in Week 2. What Channel Airs the NFL Games Tonight? What is the Monday Night Football channel for tonight's first and second matchups? The Saints and Panthers will go head to head on ESPN and ESPN2 at 7:15 p.m. ET, while the Browns travel to Pittsburgh to take on the Steelers on ABC and ESPN+ at 8:15 p.m. ET. Game Time: 7 p.m. ET. Channels: FOX, NFL+. All eyes will be on the backfields in this one. The Atlanta Falcons unveil the No. 8 overall selection in the 2023 NFL Draft, RB Bijan Robinson — widely considered the best back to enter the league in 

In [8]:
from langchain.tools import YahooFinanceNewsTool
#i need latest news on the given topic

news = YahooFinanceNewsTool("AAPL")

TypeError: Serializable.__init__() takes 1 positional argument but 2 were given

In [23]:
# from langchain.chains.summarize import load_summarize_chain
# from langchain.chat_models import ChatOpenAI
# from langchain.document_loaders import WebBaseLoader
# from langchain_core.output_parsers import StrOutputParser
# from langchain.tools import DuckDuckGoSearchRun
# from langchain.prompts import PromptTemplate

# #read syntax from syntax.md
# syntax = ""
# with open("syntax.md", "r") as f:
#     syntax = f.read()

# generate_blog_template = """you are a SEO specialist and you need to write a blog post from the following notes:
#             {notes} 
#             blog should follow the following syntax:
#             {syntax}"""

# def generate_article(url, prompt="summarize the article don't miss any important details or dates", model_name="gpt-3.5-turbo-1106"):
#     loader = WebBaseLoader(url)
#     docs = loader.load()
#     llm = ChatOpenAI(temperature=0, model_name=model_name)
#     summary_chain = load_summarize_chain(llm, chain_type="stuff")

#     chain = summary_chain  
#     return chain.invoke({"input_documents": docs, "query": prompt})





# summerize_article("https://www.cnbc.com/2021/06/11/elon-musk-says-tesla-will-most-likely-start-accepting-bitcoin-again.html")

SyntaxError: incomplete input (121342563.py, line 13)

In [1]:
from langchain.chains.summarize import load_summarize_chain
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain.tools import DuckDuckGoSearchRun
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough


url = "https://appleinsider.com/articles/23/05/31/why-android-users-have-been-switching-to-iphone-in-greater-numbers"
#read syntax from syntax.md
with open("syntax.json", "r") as f:
    syntax = f.read()

generate_blog_template = PromptTemplate(
    template="""you are a SEO specialist and you need to write a blog post from the following notes: {notes}
    blog should follow the following syntax:{syntax}
    strictly follow the format of the blog post, you can also add any markdown syntax you want to make the blog post look better
    you also have an image generator that can generate images for you if you give it a prompt
    content under each heading should also be very detailed and long and seo optimized""",
    input_variables=["notes", "syntax"]
)
summerize_blog_template = PromptTemplate(template="""you are a SEO specialist and you need to summerize the article don't miss any important details or dates""",input_variables=[])

#loader
loader = WebBaseLoader(url)
docs = loader.load()

#model
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-1106")

#chain
summary_chain = load_summarize_chain(llm, chain_type="stuff")

# ...

# chain
chain = (
    summary_chain
    | (lambda dictionary: {"notes": dictionary["output_text"], "syntax": syntax})
    | generate_blog_template
    | llm
    | StrOutputParser()
)

# invoke
blog_text = chain.invoke({"input_documents": docs, "query": summerize_blog_template})

#parse to json
import json
blog_json = json.loads(blog_text)



In [5]:
from langchain.utilities.dalle_image_generator import DallEAPIWrapper

image_url = DallEAPIWrapper(model="dall-e-3").run("A collage of smartphone users from different backgrounds, representing the diverse factors driving the shift from Android to iPhone")

image_url

#download image and save it to file
import requests
from PIL import Image
from io import BytesIO

response = requests.get(image_url)
img = Image.open(BytesIO(response.content))
img.save("image.png")


RateLimitError: Error code: 429 - {'error': {'code': 'rate_limit_exceeded', 'message': 'Rate limit exceeded for images per minute in organization org-nlfySDvgmWd6TFlDPnjMLg4X. Limit: 0/1min. Current: 1/1min. Please visit https://platform.openai.com/docs/guides/rate-limits to learn how to increase your rate limit.', 'param': None, 'type': 'requests'}}

In [8]:
from duckduckgo_search import ddg_images

#search for images
images = ddg_images("An infographic illustrating the diverse factors behind the transition of Android users to the iPhone, including previous phone issues, desire for new phone features, cost considerations, and community connectivity",license_image="ShareCommercially")

#download image and save it to file
import requests
from PIL import Image
from io import BytesIO

response = requests.get(images[0]["image"])
img = Image.open(BytesIO(response.content))
img.save("image.png")


/media/arya/aryanCodes3/article.ai/venv/lib/python3.11/site-packages/duckduckgo_search/compat.py:40: UserWarning: ddg_images is deprecated. Use DDGS().images() generator
  warnings.warn("ddg_images is deprecated. Use DDGS().images() generator")


UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x7fbea13525c0>

In [28]:


from duckduckgo_search import DDGS
import urllib.request

# search for images
images_generator = DDGS().images("Indian-CEOs-Contributions-Image", license_image="ShareCommercially")

# download image and save it to file

for image in images_generator:
    urllib.request.urlretrieve(image["image"], "image.png")
    break




    


In [1]:
from langchain.chains.summarize import load_summarize_chain
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
import json

class BlogGenerator:
    def __init__(self, syntax_file_path="syntax.json"):
        self.syntax_file_path = syntax_file_path
        self.load_syntax()

    def load_syntax(self):
        with open(self.syntax_file_path, "r") as f:
            self.syntax = f.read()

    def generate_blog_from_url(self, url, generate_prompt_template, summarize_prompt_template):
        loader = WebBaseLoader(url)
        docs = loader.load()

        llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-1106")

        summary_chain = load_summarize_chain(llm, chain_type="stuff")

        generate_blog_template = PromptTemplate(
            template=generate_prompt_template,
            input_variables=["topic", "syntax"]
        )

        summarize_blog_template = PromptTemplate(
            template=summarize_prompt_template,
            input_variables=[]
        )

        chain = (
            summary_chain
            | (lambda dictionary: {"topic": dictionary["output_text"], "syntax": self.syntax})
            | generate_blog_template
            | llm
            | StrOutputParser()
        )

        blog_text = chain.invoke({"input_documents": docs, "query": summarize_blog_template})

        return json.loads(blog_text)

    def generate_blog_from_topic(self, topic, generate_prompt_template):
        llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-1106")


        generate_blog_template = PromptTemplate(
            template=generate_prompt_template,
            input_variables=["topic", "syntax"]
        )


        chain = (
            generate_blog_template
            | llm
            | StrOutputParser()
        )

        blog_text = chain.invoke({"topic": topic, "syntax": self.syntax})

        return json.loads(blog_text)

if __name__ == "__main__":
    syntax_file_path = "syntax.json"
    generate_prompt_url = """- you are a SEO specialist and you need to write a blog post on the following notes: {topic}
        - you can add details from your own knowledge 
        - use your knowledge of SEO to write a blog post that will rank high on google
        - blog should follow the following syntax: {syntax}
        - strictly follow the format of the blog post, you can also add any markdown syntax you want to make the blog post look better
        - content under each heading should also be very detailed and long and SEO optimized"""
    
    generate_prompt_topic = """- you are a SEO specialist and you need to write a blog post on the following topic: {topic}
        - use your knowledge include data from your own knowledge 
        - use your knowledge of SEO to write a blog post that will rank high on google
        - blog should follow the following syntax: {syntax}
        - strictly follow the format of the blog post, you can also add any markdown syntax you want to make the blog post look better
        - content under each heading should also be very detailed and long and SEO optimized"""

    summarize_prompt = """you are a SEO specialist and you need to summarize the article don't miss any important details or dates"""

    blog_generator = BlogGenerator(syntax_file_path)

    # Generate blog from topic
    topic = "The Impact of Artificial Intelligence on Modern Business"
    blog_json_topic = blog_generator.generate_blog_from_topic(topic, generate_prompt_topic)
    print(blog_json_topic)


    # Generate blog from URL
    url = "https://www.indiatoday.in/education-today/gk-&-current-affairs/story/indian-origin-ceos-leading-top-companies-across-the-world-1893342-2021-12-28"
    blog_json_url = blog_generator.generate_blog_from_url(url, generate_prompt_url, summarize_prompt)
    print(blog_json_url)



    #save blogs to file
    with open("blog_url.json", "w") as f:
        json.dump(blog_json_url, f)

    with open("blog_topic.json", "w") as f:
        json.dump(blog_json_topic, f)


{'title': 'The Impact of Artificial Intelligence on Modern Business', 'date': '2022-10-15', 'tags': ['Artificial Intelligence', 'Modern Business', 'AI in Business'], 'title_image': 'A futuristic image of a robot working alongside humans in a modern office setting, showcasing the integration of AI in business operations.', 'description': 'Explore the profound impact of artificial intelligence on modern businesses and how it is revolutionizing operations, customer experiences, and decision-making processes.', 'content': [{'heading': 'Understanding Artificial Intelligence', 'image': 'An illustration of a neural network with interconnected nodes, representing the concept of AI.', 'content': 'Artificial Intelligence (AI) refers to the simulation of human intelligence processes by machines, particularly computer systems. These processes include learning, reasoning, problem-solving, perception, and language understanding. AI technologies, such as machine learning, natural language processing,

In [34]:
from langchain.chains.summarize import load_summarize_chain
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
import json

class BlogGenerator:
    """
    A class for generating blog content using ChatOpenAI and summarization chains.

    Args:
        syntax_file_path (str): The file path to the syntax template.
    """
    def __init__(self, syntax_file_path="syntax.json"):
        self.syntax_file_path = syntax_file_path
        self.load_syntax()

    def load_syntax(self):
        """
        Load the syntax template from the specified file.
        """
        try:
            with open(self.syntax_file_path, "r") as f:
                self.syntax = f.read()
        except FileNotFoundError:
            raise FileNotFoundError(f"Syntax file not found at path: {self.syntax_file_path}")

    def generate_blog_from_url(self, url, generate_prompt_template=None, summarize_prompt_template=None):
        """
        Generate a blog from a given URL.

        Args:
            url (str): The URL of the article.
            generate_prompt_template (PromptTemplate, optional): The template for generating content.
            summarize_prompt_template (PromptTemplate, optional): The template for summarizing content.

        Returns:
            dict: The generated blog content in JSON format.
        """
        try:
            loader = WebBaseLoader(url)
            docs = loader.load()

            llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-1106")

            # Use default prompts if none are provided
            if generate_prompt_template is None:
                generate_prompt_template = PromptTemplate(
                    template="""- you are a SEO specialist and you need to write a blog post on the following notes: {topic}
                        - you can add details from your own knowledge 
                        - use your knowledge of SEO to write a blog post that will rank high on google
                        - blog should follow the following syntax: {syntax}
                        - strictly follow the format of the blog post, you can also add any markdown syntax you want to make the blog post look better
                        - content under each heading should also be very detailed and long and SEO optimized""",
                    input_variables=["topic", "syntax"]
                )

            if summarize_prompt_template is None:
                summarize_prompt_template = PromptTemplate(
                    template="""you are a SEO specialist and you need to summarize the article don't miss any important details or dates""",
                    input_variables=[]
                )

            summary_chain = load_summarize_chain(llm, chain_type="stuff")

            chain = (
                summary_chain
                | (lambda dictionary: {"topic": dictionary["output_text"], "syntax": self.syntax})
                | generate_prompt_template
                | llm
                | StrOutputParser()
            )

            blog_text = chain.invoke({"input_documents": docs, "query": summarize_prompt_template})

            return json.loads(blog_text)

        except Exception as e:
            raise RuntimeError(f"Error generating blog from URL: {str(e)}")

    def generate_blog_from_topic(self, topic, generate_prompt_template=None):
        """
        Generate a blog from a given topic.

        Args:
            topic (str): The topic of the blog post.
            generate_prompt_template (PromptTemplate, optional): The template for generating content.

        Returns:
            dict: The generated blog content in JSON format.
        """
        try:
            llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-1106")

            # Use default prompt if none is provided
            if generate_prompt_template is None:
                generate_prompt_template = PromptTemplate(
                    template="""- you are a SEO specialist and you need to write a blog post on the following topic: {topic}
                        - use your knowledge include data from your own knowledge 
                        - use your knowledge of SEO to write a blog post that will rank high on google
                        - blog should follow the following syntax: {syntax}
                        - strictly follow the format of the blog post, you can also add any markdown syntax you want to make the blog post look better
                        - content under each heading should also be very detailed and long and SEO optimized""",
                    input_variables=["topic", "syntax"]
                )

            chain = (
                generate_prompt_template
                | llm
                | StrOutputParser()
            )

            blog_text = chain.invoke({"topic": topic, "syntax": self.syntax})

            return json.loads(blog_text)

        except Exception as e:
            raise RuntimeError(f"Error generating blog from topic: {str(e)}")

if __name__ == "__main__":
    syntax_file_path = "syntax.json"

    blog_generator = BlogGenerator(syntax_file_path)

    # Generate blog from topic with default prompts
    topic = "The Impact of Artificial Intelligence on Modern Business"
    try:
        blog_json_topic = blog_generator.generate_blog_from_topic(topic)
        print(blog_json_topic)
    except Exception as e:
        print(f"Error generating blog from topic: {str(e)}")

    # Generate blog from URL with default prompts
    url = "https://www.indiatoday.in/education-today/gk-&-current-affairs/story/indian-origin-ceos-leading-top-companies-across-the-world-1893342-2021-12-28"
    try:
        blog_json_url = blog_generator.generate_blog_from_url(url)
        print(blog_json_url)
    except Exception as e:
        print(f"Error generating blog from URL: {str(e)}")

    # Save blogs to file
    try:
        with open("blog_url.json", "w") as f:
            json.dump(blog_json_url, f)

        with open("blog_topic.json", "w") as f:
            json.dump(blog_json_topic, f)
    except Exception as e:
        print(f"Error saving blogs to file: {str(e)}")


{'title': 'The Impact of Artificial Intelligence on Modern Business', 'date': '2022-10-15', 'tags': ['Artificial Intelligence', 'Modern Business', 'AI in Business'], 'title_image': 'The Rise of Artificial Intelligence in Business: How AI is Transforming Modern Enterprises', 'description': 'Explore the profound impact of artificial intelligence on modern business and how it is revolutionizing the way companies operate and compete in the digital age.', 'content': [{'heading': 'Understanding Artificial Intelligence', 'image': 'Artificial Intelligence in Business', 'content': 'Artificial Intelligence (AI) refers to the simulation of human intelligence processes by machines, particularly computer systems. This includes learning, reasoning, problem-solving, perception, and language understanding. In the context of modern business, AI technologies are being used to automate repetitive tasks, analyze data at scale, and make predictions based on patterns and trends.'}, {'heading': 'Enhancing Bu

In [37]:
import os
import urllib.request
from duckduckgo_search import DDGS
import json
from time import sleep

class ImageExtractor:
    def __init__(self, json_data, output_dir):
        """
        Initializes the ImageExtractor with JSON data and output directory.

        Args:
            json_data (dict): The JSON data containing information about the images.
            output_dir (str): The directory where the images will be saved.
        """
        self.json_data = json_data
        self.output_dir = output_dir
        self.retry_attempts = 3  # Number of retry attempts

    def search_and_download_images(self, query):
        """
        Search for images using DuckDuckGo and download the first image.

        Args:
            query (str): The search query for images.

        Returns:
            str: The path to the downloaded image.
        """
        for attempt in range(self.retry_attempts):
            try:
                # Create the output directory if it doesn't exist
                os.makedirs(self.output_dir, exist_ok=True)

                # Search for images
                images_generator = DDGS().images(query, license_image="ShareCommercially")

                # Download and save the first image
                for image in images_generator:
                    image_path = os.path.join(self.output_dir, f"{query.replace(' ', '_')}_image.png")
                    urllib.request.urlretrieve(image["image"], image_path)
                    print(f"Image downloaded: {image_path}")
                    return image_path  # Only download the first image
            except Exception as e:
                print(f"Error searching and downloading images (attempt {attempt + 1}/{self.retry_attempts}): {str(e)}")
                sleep(5)  # Sleep for 5 seconds before retrying
        raise RuntimeError(f"Failed to download image after {self.retry_attempts} attempts.")

    def download_images(self):
        """
        Download images from the provided JSON data and save them to the output directory.
        """
        try:
            # Download title_image using the search function
            title_image_url = self.search_and_download_images(self.json_data.get("title_image", ""))
            if title_image_url:
                print(f"Title image downloaded: {title_image_url}")

            # Download images from content
            for content_item in self.json_data.get("content", []):
                image_url = self.search_and_download_images(content_item.get("image", ""))
                if image_url:
                    print(f"Image downloaded: {image_url}")
                sleep(5)  # Sleep for 5 seconds to avoid rate limiting

        except Exception as e:
            raise RuntimeError(f"Error downloading images: {str(e)}")


if __name__ == "__main__":
    # Your JSON data
    # Load json from file
    with open("blog_topic.json", "r") as f:
        json_data = json.load(f)

    # Output directory for saving images
    output_dir = "images_output"

    # Create an instance of ImageExtractor
    image_extractor = ImageExtractor(json_data, output_dir)

    try:
        # Download and save images
        image_extractor.download_images()
    except Exception as e:
        print(f"Error in image extraction: {str(e)}")


Image downloaded: images_output/The_Rise_of_Artificial_Intelligence_in_Business:_How_AI_is_Transforming_Modern_Enterprises_image.png
Title image downloaded: images_output/The_Rise_of_Artificial_Intelligence_in_Business:_How_AI_is_Transforming_Modern_Enterprises_image.png
Image downloaded: images_output/Artificial_Intelligence_in_Business_image.png
Image downloaded: images_output/Artificial_Intelligence_in_Business_image.png
Image downloaded: images_output/AI_Automation_in_Business_image.png
Image downloaded: images_output/AI_Automation_in_Business_image.png
Image downloaded: images_output/AI_Data_Analysis_in_Business_image.png
Image downloaded: images_output/AI_Data_Analysis_in_Business_image.png
Image downloaded: images_output/AI_Customer_Personalization_image.png
Image downloaded: images_output/AI_Customer_Personalization_image.png
Image downloaded: images_output/AI_Challenges_and_Opportunities_image.png
Image downloaded: images_output/AI_Challenges_and_Opportunities_image.png
